### Connect to SQL in GCP

In [83]:
from google.cloud.sql.connector import Connector, IPTypes
import pg8000
import sqlalchemy
import pandas as pd
import time


connection ='w210-capstone-ta:us-central1:articles'
region = 'us-central1'
user = 'postgres'
password = 'V>4;MYzd&.n^B&2%'
name = 'article_attributes'
ip_private = True

# connect to google cloud postgres db
def connect_with_connector() -> sqlalchemy.engine.base.Engine:

    instance_connection_name = connection 
    db_user = user
    db_pass = password
    db_name = name

    ip_type = IPTypes.PRIVATE 
    
    
    # initialize Cloud SQL Python Connector object
    connector = Connector()

    def getconn() -> pg8000.dbapi.Connection:
        conn: pg8000.dbapi.Connection = connector.connect(
            instance_connection_name,
            "pg8000",
            db=db_name,
            user=db_user,
            password = db_pass,
            # enable_iam_auth=True,
            ip_type=ip_type
        )
        return conn

    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
        pool_size=5,
        max_overflow=2,
        pool_timeout=30,  
        pool_recycle=1800,  
    )
    return pool

pool = connect_with_connector()
pool.connect()
pd.set_option('display.max_colwidth', None)

### Check data

In [87]:
pd.read_sql_query(""" 

select *
from article_attributes
limit 1


""",pool)

,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,2842385,"{""ACL"": null, ""DBLP"": ""conf/cogsci/DamenWAK17"", ""ArXiv"": null, ""MAG"": ""2890935122"", ""CorpusId"": ""2842385"", ""PubMed"": ""30219958"", ""DOI"": ""10.1007/s10936-018-9602-7"", ""PubMedCentral"": ""6426803""}",https://www.semanticscholar.org/paper/2b1114c8bbac93c1df521a9cd5253335972ba2fa,Perspective-Taking in Referential Communication: Does Stimulated Attention to Addressees’ Perspective Influence Speakers’ Reference Production?,"{""{\""authorId\"": \""32592077\"", \""name\"": \""Debby Damen\""}"",""{\""authorId\"": \""2946360\"", \""name\"": \""P. V. D. Wijst\""}"",""{\""authorId\"": \""2583854\"", \""name\"": \""Marije van Amelsvoort\""}"",""{\""authorId\"": \""145210073\"", \""name\"": \""E. Krahmer\""}""}",Annual Meeting of the Cognitive Science Society,9c06885c-ecb6-4a76-ba1e-fbad53521efd,2017,69,7,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \""s2-fos-model\""}"",""{\""category\"": \""Psychology\"", \""source\"": \""external\""}"",""{\""category\"": \""Medicine\"", \""source\"": \""external\""}"",""{\""category\"": \""Computer Science\"", \""source\"": \""external\""}""}",{JournalArticle},2017-07-01,"{""name"": ""Journal of Psycholinguistic Research"", ""pages"": ""257 - 288"", ""volume"": ""48""}",2023-03-31T01:17:13.414Z,5692595


In [57]:
pd.read_sql_query(""" 

select *
from article_attributes
limit 1


""",pool).columns

Index(['corpusid', 'externalids', 'url', 'title', 'authors', 'venue',
       'publicationvenueid', 'year', 'referencecount', 'citationcount',
       'influentialcitationcount', 'isopenaccess', 's2fieldsofstudy',
       'publicationtypes', 'publicationdate', 'journal', 'updated', 'id'],
      dtype='object')

### Check for dups and null values

In [3]:
## 13,507,662 duplicated input rows
pd.read_sql_query('''

select corpusid
from article_attributes
group by corpusid
having count(*) > 1


''',pool)

,corpusid
0,8101124
1,126327610
2,232809352
3,135664274
4,255454848
...,...
13507657,157226250
13507658,188756524
13507659,195079922
13507660,10489205


**Display Dupped Row**

In [33]:
#display first dupped record

pd.read_sql_query("""

select *
from article_attributes a,
article_attributes b
where a.corpusid = b.corpusid
limit 4

""",pool)

,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,...,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,0,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""0"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",None,None,None,None,None,None,0,0,...,0,0,0,False,None,None,None,None,None,4098943
1,0,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""0"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",None,None,None,None,None,None,0,0,...,0,0,0,False,None,None,None,None,None,4317542
2,0,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""0"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",None,None,None,None,None,None,0,0,...,0,0,0,False,None,None,None,None,None,4098943
3,0,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""0"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",None,None,None,None,None,None,0,0,...,0,0,0,False,None,None,None,None,None,4317542


**Check row where field of study = None**

In [11]:
#None in field of study will not parse in next step, no need to remove

pd.read_sql_query('''

select *
from article_attributes
where corpusid = 63 

''',pool)

,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,63,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""63"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",https://www.semanticscholar.org/paper/a786ef5ef7cfcbb9ded60995107bddf4dcbe2187,Southwest Airlines and the Impact of Low-Cost Carriers On Airline Ticket Prices,"{""{\""authorId\"": \""2067198527\"", \""name\"": \""Sidney Field\""}""}",,None,2016,14,3,0,False,None,None,None,None,2022-09-04T14:25:52.360Z,13293209
1,63,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""63"", ""PubMed"": null, ""DOI"": null, ""PubMedCentral"": null}",https://www.semanticscholar.org/paper/a786ef5ef7cfcbb9ded60995107bddf4dcbe2187,Southwest Airlines and the Impact of Low-Cost Carriers On Airline Ticket Prices,"{""{\""authorId\"": \""2067198527\"", \""name\"": \""Sidney Field\""}""}",,None,2016,14,3,0,False,None,None,None,None,2022-09-04T14:25:52.360Z,13314134


**Count null values in field of study**

In [38]:
#10.7mm null field of study

pd.read_sql_query('''

select count(*)
from article_attributes
where s2fieldsofstudy is null


''',pool)

,count
0,10746037


### Create Index on Corpus ID

In [ ]:
## index corpusid to speed up look ups


# pd.read_sql_query("""

# create index on article_attributes (corpusid)

# """,pool)

### Create Primary Key to remove dups

In [ ]:
## added a key to be able to reference in the delete statement

# pd.read_sql_query('''

# ALTER TABLE article_attributes ADD COLUMN id SERIAL PRIMARY KEY;

# ''',pool)

In [14]:
## todo: need to delete duplicate rows while keeping first row in order to apply primary key
#display first dupped record
DELETE FROM Customers WHERE CustomerName='Alfreds Futterkiste';

pd.read_sql_query("""

select b.corpusid
from article_attributes a,
article_attributes b
where a.corpusid = b.corpusid and a.id < b.id
limit 5 

""",pool)


,corpusid
0,0
1,23
2,25
3,50
4,63


In [25]:
#corpusid	externalids	url	title	authors	venue	publicationvenueid	year	
#referencecount	citationcount	influentialcitationcount	isopenaccess	s2fieldsofstudy	publicationtypes
#publicationdate	journal	updated	id

pd.read_sql_query(""" 

with fields_of_study_multirow as

(select corpusid, externalids, url, title, authors, venue, publicationvenueid, year,
referencecount, citationcount, influentialcitationcount, isopenaccess, s2fieldsofstudy, publicationtypes,
publicationdate, journal, updated


from article_attributes
group by 1, 2, 3
limit 1000)

select *
from fields_of_study_multirow
where corpusid = 23

""",pool)




,corpusid,externalids,url
0,23,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MAG"": null, ""CorpusId"": ""23"", ""PubMed"": ""14901052"", ""DOI"": null, ""PubMedCentral"": null}",https://www.semanticscholar.org/paper/e975952400b4f97e9b6958b36434e6b680d5042c


**Pull Full DB Row**

**Full DB Analysis - Fields of Study S2 model**

In [30]:
seconds = time.time()

x = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, 
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes),

fields_of_study_detail as

(select corpusid,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow
where field_of_study->> 'source' = 's2-fos-model')

select field_of_study_category, count(field_of_study_category)
from fields_of_study_detail
group by 1


''',pool)


print("Seconds since query =", time.time() - seconds)


x['%'] = x['count'] / x['count'].sum()
x = x.sort_values(by = '%', ascending = False)
x


Seconds since query = 141.75292301177979


,field_of_study_category,count,%
17,Medicine,4268939,0.214732
2,Biology,2563502,0.128947
19,Physics,1487712,0.074834
5,Computer Science,1223533,0.061545
4,Chemistry,1186357,0.059675
8,Engineering,1137550,0.057220
15,Materials Science,922735,0.046415
7,Education,855795,0.043047
21,Psychology,814435,0.040967
9,Environmental Science,796014,0.040040


In [31]:
x['count'].sum()

19880283

**Full DB Analysis - Fields of Study external validation**

In [32]:
seconds = time.time()

x1 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, 
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes),

fields_of_study_detail as

(select corpusid,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow
where field_of_study->> 'source' = 'external')

select field_of_study_category, count(field_of_study_category)
from fields_of_study_detail
group by 1


''',pool)


print("Seconds since query =", time.time() - seconds)


x1['%'] = x1['count'] / x1['count'].sum()
x1 = x1.sort_values(by = '%', ascending = False)
x1

Seconds since query = 130.48064184188843


,field_of_study_category,count,%
13,Medicine,6059924,0.308271
4,Computer Science,1733640,0.088191
3,Chemistry,1702433,0.086604
1,Biology,1675183,0.085217
11,Materials Science,1323228,0.067313
6,Engineering,1083006,0.055093
15,Physics,902060,0.045888
17,Psychology,888996,0.045224
12,Mathematics,698244,0.035520
16,Political Science,530143,0.026969


In [34]:
x1['count'].sum()

19657761

### No Filtering

**Rows no filter** 

In [40]:
x0 = pd.read_sql_query('select count(corpusid) from article_attributes', pool)
x0

,count
0,27961430


### Filter 1

**Filter So2 model / external validation subject same, remove if they are not the same or null**

In [47]:
seconds = time.time()

x3 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, 
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source
from external
join field_of_source_s2 using (corpusid))

select count(*)
from joined_table
where external_field_of_study_category = field_of_study_category

''',pool)


print("Seconds since query =", time.time() - seconds)

x3



Seconds since query = 191.85204243659973


,count
0,21905085


### Filter 2 

**Filter 1 + Filter for relevant subjects** 

In [48]:
seconds = time.time()

x4 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, 
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source
from external
join field_of_source_s2 using (corpusid))

select count(*)
from joined_table
where external_field_of_study_category = field_of_study_category
and field_of_study_category in ('Chemistry', 'Biology', 'Psychology', 'Medicine')


''',pool)


print("Seconds since query =", time.time() - seconds)

x4


Seconds since query = 172.4147024154663


,count
0,13038613


### Filter 3

**Filter 1, 2 + Filter for arabic letters**

In [51]:
seconds = time.time()

x5 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, title, 
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid, title,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source, title
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source, title
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source,
external.title
from external
join field_of_source_s2 using (corpusid))

select count(*)
from joined_table
where external_field_of_study_category = field_of_study_category
and field_of_study_category in ('Chemistry', 'Biology', 'Psychology', 'Medicine')
and title ~ '[A-Za-z]+$'

''',pool)


print("Seconds since query =", time.time() - seconds)

x5

Seconds since query = 235.9822380542755


,count
0,7136528


### Filter 4

**Filter 1, 2, 3 + Filter for atleast 1 reference**

In [52]:
seconds = time.time()

x6 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, title, referencecount,
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid, title, referencecount,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source, title, referencecount
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source, title, referencecount
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source,
external.title, external.referencecount
from external
join field_of_source_s2 using (corpusid))

select count(*)
from joined_table
where external_field_of_study_category = field_of_study_category
and field_of_study_category in ('Chemistry', 'Biology', 'Psychology', 'Medicine')
and title ~ '[A-Za-z]+$'
and referencecount > 1

''',pool)


print("Seconds since query =", time.time() - seconds)

x6

Seconds since query = 226.173757314682


,count
0,4423236


### Filter 5

**Filter 1-4 + Filter for atleast 1 citation**

In [71]:
seconds = time.time()

x7 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, title, referencecount, citationcount,
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid, title, referencecount, citationcount,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source, title, 
referencecount, citationcount
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source, title, 
referencecount, citationcount
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source,
external.title, external.referencecount, external.citationcount
from external
join field_of_source_s2 using (corpusid))

select count(*)
from joined_table
where external_field_of_study_category = field_of_study_category
and field_of_study_category in ('Chemistry', 'Biology', 'Psychology', 'Medicine')
and title ~ '[A-Za-z]+$'
and referencecount > 1
and citationcount > 1

''',pool)


print("Seconds since query =", time.time() - seconds)

x7


Seconds since query = 228.62232971191406


,count
0,3396448


### Filter 6

**Filter 1-5 + Filter for dups**

In [75]:
seconds = time.time()

x8 = pd.read_sql_query('''

with fields_of_study_multirow as

(select corpusid, title, referencecount, citationcount,
    unnest(s2fieldsofstudy::text[])::json as field_of_study
from article_attributes
where s2fieldsofstudy is not null),

fields_of_study_detail as

(select corpusid, title, referencecount, citationcount,
    field_of_study->> 'category' as field_of_study_category,
    field_of_study->> 'source' as field_of_study_source
from fields_of_study_multirow),

field_of_source_s2 as

(select corpusid, field_of_study_category, field_of_study_source, title, 
referencecount, citationcount
from fields_of_study_detail
where field_of_study_source = 's2-fos-model'),

external as

(select corpusid, field_of_study_category, field_of_study_source, title, 
referencecount, citationcount
from fields_of_study_detail
where field_of_study_source = 'external'),

joined_table as

(select external.corpusid, external.field_of_study_category as external_field_of_study_category,
external.field_of_study_source as external_field_of_study_source,
field_of_source_s2.field_of_study_category, field_of_source_s2.field_of_study_source,
external.title, external.referencecount, external.citationcount
from external
join field_of_source_s2 using (corpusid)),

filtered_table as 

(select corpusid
from joined_table
where external_field_of_study_category = field_of_study_category
and field_of_study_category in ('Chemistry', 'Biology', 'Psychology', 'Medicine')
and title ~ '[A-Za-z]+$'
and referencecount > 1
and citationcount > 1)

select corpusid
from filtered_table
group by 1


''',pool)


print("Seconds since query =", time.time() - seconds)

x8


Seconds since query = 259.44938802719116


,corpusid
0,25
1,50
2,143
3,939
4,1217
...,...
729812,258462340
729813,258463542
729814,258505974
729815,258714040


In [78]:
x8['corpusid'].duplicated().any()

False

In [80]:
x8.to_csv('final_corpuslist.csv') 

In [113]:
t = tuple(x8['corpusid'])
query = "select * from article_attributes where corpusid IN {}".format(t)

In [116]:
final.shape

(1509105, 18)

In [115]:
final = pd.read_sql_query(query,pool)
print (final.head(3))

   corpusid  \
0  52273203   
1  23708908   
2  13232625   

                                                                                                                                                                     externalids  \
0  {"ACL": null, "DBLP": null, "ArXiv": null, "MAG": "2889324973", "CorpusId": "52273203", "PubMed": "30272009", "DOI": "10.1038/s42003-018-0133-4", "PubMedCentral": "6123708"}   
1  {"ACL": null, "DBLP": null, "ArXiv": null, "MAG": "2765137431", "CorpusId": "23708908", "PubMed": "29228628", "DOI": "10.18632/oncotarget.22178", "PubMedCentral": "5722580"}   
2                      {"ACL": null, "DBLP": null, "ArXiv": null, "MAG": "1993137580", "CorpusId": "13232625", "PubMed": null, "DOI": "10.1002/ACP.1751", "PubMedCentral": null}   

                                                                              url  \
0  https://www.semanticscholar.org/paper/39063bcddbd9decd4c8f5c5d3e8392280ab046ad   
1  https://www.semanticscholar.org/paper/6ec7c15

In [117]:
final.to_csv('final.csv') 

In [118]:
from semanticscholar import SemanticScholar
sch = SemanticScholar()


sch = SemanticScholar(api_key='jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB')

seconds = time.time()

paper = sch.get_paper(['CorpusId:85050054','CorpusId:85050053'])  #'10.1093/mind/lix.236.433')

print (time.time() - seconds)
  #'10.1093/mind/lix.236.433')

paper.referenceCount #number of references in this one, can help us create some sort of pagerank type algo
paper.citationCount #number of papers that cited this one
paper.externalIds 
paper.abstract



JSONDecodeError: Expecting value: line 1 column 1 (char 0)